# importazione librerie

In [1]:
import pandas as pd
#import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt 
import folium 
import math
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances,haversine_distances
from typing import List
from collections import defaultdict 
from tqdm.notebook import tqdm
import networkx as nx
from math import radians
import json

# importazione dataset e caratterizzazione dati

In [2]:
#Read the dataset 

fields = ["city","lat","lng","country","population"]
df = pd.read_excel('worldcities.xlsx', usecols=fields,engine="openpyxl")
airports_df= pd.read_csv('airports.csv')
Paths_df= pd.read_csv('routes.csv')
number_city = len(df)
#sostituisco il nome della prima colonna del data frame airport_df per comodità (tolgo lo spazio)
aux_list = list(airports_df.columns)
aux_list[0] = 'Airport_ID'
airports_df.columns = aux_list
#elimino gli spazi dai nomi delle colonne di Paths_df
Paths_df.columns =[column.replace(" ", "_") for column in Paths_df.columns]
#caratterizzo i dati 
#elimino da paths_df tutte le colonne che hanno dati rilevanti mancanti
Paths_df = Paths_df[(Paths_df['Destination_airport_ID'] != '\\N') & (Paths_df['Source_airport_ID'] != '\\N')]
#converto i dati nelle colonne "Source airport ID", "Destination airport ID" in interi
Paths_df[["Source_airport_ID", "Destination_airport_ID"]] = Paths_df[["Source_airport_ID", "Destination_airport_ID"]].apply(pd.to_numeric)
#creo le colonne 'Latitude_rad' 'Longitude_rad' applicando la funzione radians(x) alle colonne "Latitude" "Longitude"
#questo per evitare in futuro di doverle calcolare più volte inutilmente
airports_df['Latitude_rad'] = airports_df["Latitude"].apply(lambda x: radians(x))
airports_df['Longitude_rad'] = airports_df["Longitude"].apply(lambda x: radians(x))
df['lat_rad'] = df['lat'].apply(lambda x: radians(x))
df['lng_rad'] = df['lng'].apply(lambda x: radians(x))

# creazioni funzioni richiamabili ausiliarie

In [3]:
def int_to_list (a):
    #converte un intero in una lista con quell'intero come unico elemento
    if type (a) == int:
        return([a])
    else:
        return(a)
def List_comparison (a,b):
    #prende in input due liste, se hanno almeno un elemento in comune restituisce True, altrimenti False
    result = False
    for e in a:
        if e in b:
            result = True
    return(result)

# passo 1 associazione per ogni città degli aereoporti corrispondenti o limitrofi

In [4]:
#Passo 1.1 versione "bradipo"
#trovo il codice dell'aeroporto della corrispettiva città, se sono presenti più aeroporti per una città vengono 
#elencati in una lista
#l'associazione è fatta per le coppie (city,country) , non posso associare solamente usanto city come discriminante
#in quanto, per esempio, possono esserci due città con lo stesso nome (Valencia esiste sia in Spagna sia in Venezuela)
#è possibile che vi siano città senza alcun aereoporto associato dopo questa fase per 2 motivi:
# 1: non esiste effettivamente alcun aeroporto
# 2: i dataframe df e airports_df non hanno rappresentazioni univoce per la coppia city-country in quanto certe 
#rappresentazioni di country non sono coerenti (es: South Korea in df e Korea,South in airports_df)
lista = [[]] *(len(df.index))
for e in tqdm(df.index):
        lista[e] = list(airports_df[(airports_df.City == df.loc[e].city) & (airports_df.Country == df.loc[e].country)].Airport_ID)
df['airport_id'] = lista

In [5]:
#passo 1.1 versione ottimizzata
#l'ottimizzazione consisite nel creare un dizionario ausiliario contenente come chiavi i cretieri di ricerca che 
#mi servono in airports_df (città e paese) e come valori gli aereoporti associato

lista = [[]] *(len(df.index))
dictionary_aux ={}
for e in tqdm(airports_df.index):
    try:
        dictionary_aux[(airports_df.loc[e,'City'],airports_df.loc[e,'Country'])].append(airports_df.loc[e,'Airport_ID'])
    except KeyError:
        dictionary_aux[(airports_df.loc[e,'City'],airports_df.loc[e,'Country'])] = [airports_df.loc[e,'Airport_ID']]
        
for e in tqdm(df.index):
        try:
            lista[e] = dictionary_aux[(df.loc[e,'city'],df.loc[e,'country'])]
        except KeyError:
            lista[e] = []
df['airport_id'] = lista
#l'aumento di performance è evidente...

In [6]:
#per tutte le città senza aeroporto calcolo l'areoporto più vicino nello stesso paese
#se non esiste un aeroporto in quel paese cerco l'aeroporto più vicino
df['own_airport'] = np.ones(len(df.index))
for e in tqdm(df.index):
    if df.loc[e,'airport_id'] ==[]:
        country = df.loc[e].country
        #aux_airports contiene tutti gli aeroporti presenti nel paese della città che sto analizzando
        aux_airports = airports_df[airports_df.Country == country]
        #per ogni aeroporto calcolo la distanza dalla città che sto analizzando
        aux_airports['distance'] = [haversine_distances([[df.loc[e,'lat_rad'],df.loc[e,'lng_rad']],
                                                     [aux_airports.loc[x,'Latitude_rad'],
                                                      aux_airports.loc[x,'Longitude_rad']]])[0][1]
                                for x in aux_airports.index]
        #i dataframe airports_df e df presentano alcune incongruenze sulla stringa che definisce la nazione,
        #è quindi possibile che aux_airports non contenga alcun record proprio per questa incongruenza
        #per ovviare al problema è stato realizzata una gestione dell'eccezione che va a cercare
        #l'aereoporto più vicino in termini assoluti
    
        try:
            df.loc[e,'airport_id'] = int(aux_airports[aux_airports.distance ==  aux_airports['distance'].min()].Airport_ID)
            if df.loc[e,'city'] != aux_airports[aux_airports.distance ==  aux_airports['distance'].min()].City.item():
                df.loc[e,'own_airport'] = 0
        except TypeError:
            aux_airports = airports_df
            #per ogni aeroporto calcola la distanza da quella città e prelevo solo gli aereoporti di quella 
            #città oppure il più vicino
            aux_airports['distance'] = [haversine_distances([[df.loc[e,'lat_rad'],df.loc[e,'lng_rad']],
                                                         [aux_airports.loc[x,'Latitude_rad'],
                                                          aux_airports.loc[x,'Longitude_rad']]])[0][1]
                                    for x in aux_airports.index]
            #ordino gli aeroporti per distanza
            aux_airports = aux_airports.sort_values(by=['distance'], ascending = 0, inplace = False)
            #trovo il primo aeroporto che non appartiene a quella città
            count = 0
            find = 0
            #l'aeroporto più vicino è effettivamente associato alla città analizzata se ne esiste uno oppure 
            #potrebbe essere associato ad un altra città limitrofa
            while find == 0:
                if aux_airports.iloc[count]['City'] != df.loc[e,'city']:
                    find = 1
                else:
                    count += 1
            #se count >0 ho trovato aereoporti effettivamente associati alla città
            #se count è zero ho trovato l'aereoporto più vicino che tuttavia è in una città limitrofa e devo considerare
            #che devo fare un extra tragitto epr raggiungere l'aereoporto
            if count > 0:
                aux_airports = aux_airports.iloc[:count]
                df.loc[e,'airport_id'] = [a for a in aux_airports['Airport_ID']]
            if count == 0:
                df.loc[e,'own_airport'] = 0
                df.loc[e,'airport_id'] = aux_airports.iloc[0]['Airport_ID']

df['airport_id'] = df['airport_id'].apply(lambda x: int_to_list(x))             

<ipython-input-6-abaf2cafd64d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux_airports['distance'] = [haversine_distances([[df.loc[e,'lat_rad'],df.loc[e,'lng_rad']],


In [7]:
#salvo il dataframe ricalcolato per evitare ulteriori inizializzazioni
df.to_csv (r'export_dataframe_1.csv', index = False, header=True)

In [8]:
#riprendo il dataframe già calcolato
df = pd.read_csv('export_dataframe_1.csv')
df['airport_id'] = df['airport_id'].apply(lambda x: json.loads(x))
df['airport_id'] = df['airport_id'].apply(lambda x: int_to_list(x))

# passo 2 associazione dei tragitti per ogni città e creazione del grafo

In [9]:
#calcolo la distanza di ogni tragitto
Paths_df['path_lenght'] = np.zeros(len(Paths_df.index))
for e in tqdm(Paths_df.index):
    start = Paths_df.loc[e,'Source_airport_ID']
    end = Paths_df.loc[e,'Destination_airport_ID']
    try:
        Paths_df.loc[e,'path_lenght'] = haversine_distances([[airports_df[airports_df.Airport_ID == start].Latitude_rad.item()
                                                        ,airports_df[airports_df.Airport_ID == start].Longitude_rad.item()]
                                                        ,[airports_df[airports_df.Airport_ID == end].Latitude_rad.item()
                                                          ,airports_df[airports_df.Airport_ID == end].Longitude_rad.item()]])[0][1]

    #nel caso non ci sia alcun aeroporto correlato alla tratta,sia di partenzia che di arrivo, elimino quella tratta
    except ValueError:
        Paths_df = Paths_df.loc[Paths_df.index != e]
    

In [10]:
dictionary = {}
earth_diameter =12742 #[km]
Plane_Speed  = 900 #[km/h]
#per ogni città cerco i 30 percorsi disponibili che hanno come partenza un aereoporto di quella città ordinati per distanza
for e in tqdm(df.index):
        airports_list = df.loc[e,'airport_id']
        Paths_Available = Paths_df.query('Source_airport_ID in @airports_list').sort_values(by=['path_lenght'], 
                                                                                          ascending = 1, inplace = False).iloc[0:30]
        #ci sono dei path che presentano stessa partenza e stesso arrivo ma con modelli di aereo differenti, 
        #elimino quindi i duplicati che si possono creare per questo effetto
        #trasformando i risultati in un insieme e poi di nuovo in una lista
        Paths_Available_List = list(set([h for h in Paths_Available['Destination_airport_ID']]))
        #tra tutte le città disponibili, quelle raggiungibili sono quelle che hanno come aereoporto associato un aereoporto
        #di destinazione contenuto in Paths_Avaible_List
        City_reacheable = df
        City_reacheable['Linked'] = City_reacheable['airport_id'].apply(lambda x: List_comparison(Paths_Available_List,x))
        City_reacheable = City_reacheable[City_reacheable['Linked'] == True]
        links_list = []
        start_latutude = df.loc[e,'lat']
        start_longitude = df.loc[e,'lng']
        for t in City_reacheable.index:
            #2 ore aggiuntive se la città non ha l'aeroporto
            distance = haversine_distances([[start_latutude,start_longitude]
                                            ,[City_reacheable.loc[t,'lat']
                                              ,City_reacheable.loc[t,'lng']]])[0][1]
            time = distance * (earth_diameter/2) / Plane_Speed
            time_path = 2 * (City_reacheable.loc[t,'own_airport']+df.loc[e,'own_airport']) + time 
            links_list.append((t,time_path))
        dictionary[e] = links_list    
            

In [11]:
#salvo il dizionario per evitare ulteriori inizializzazioni
with open("dictionary.json", "w") as outfile:
    json.dump(dictionary, outfile)

In [12]:
#riprendo il dizionario già calcolato
with open("dictionary.json", "r") as outfile:
    data = json.load(outfile)
dictionary = {}
for e in data.keys():
    dictionary[int(e)] = data[e]
    

In [13]:
# creation of the graph with Networkx library from the dictionary C 
G = nx.DiGraph()
for city in dictionary:
    G.add_node(city)

for city in dictionary:
    for near_city in dictionary[city]:
        G.add_edge(city,near_city[0],weight = near_city[1])

In [14]:
nx.has_path(G,536,873)

True

In [15]:
my_path=nx.shortest_path(G,536,873, weight="weight")
#ore = nx.path_weight(G,my_path, "weight")

In [16]:
print(my_path)
#for e in my_path:
    #print(dictionary[e],'\n')
df.loc[my_path]

[536, 17805, 5325, 3659, 14372, 4403, 14239, 5750, 198, 18281, 3357, 7098, 5267, 4862, 2020, 5619, 1369, 1411, 873]


,city,lat,lng,country,population,lat_rad,lng_rad,airport_id,own_airport,Linked
536,Milan,45.4669,9.1900,Italy,1351562.0,0.793547,0.160396,[1529],1.0,False
17805,Múggia,45.6000,13.7667,Italy,13111.0,0.795870,0.240274,[1545],0.0,False
5325,Braintree,51.8780,0.5500,United Kingdom,53477.0,0.905442,0.009599,[548],0.0,False
3659,’s-Hertogenbosch,51.6833,5.3167,Netherlands,134520.0,0.902044,0.092794,[585],0.0,False
14372,Konstancin-Jeziorna,52.0938,21.1176,Poland,23694.0,0.909208,0.368572,[679],0.0,False
4403,Brovary,50.5114,30.7903,Ukraine,102856.0,0.881590,0.537392,[2939],0.0,False
14239,Zernograd,46.8500,40.3000,Russia,24561.0,0.817687,0.703368,[2964],0.0,False
5750,Umm al Qaywayn,25.5653,55.5533,United Arab Emirates,44411.0,0.446199,0.969588,[2191],0.0,False
198,Jeddah,21.5428,39.1728,Saudi Arabia,3976000.0,0.375993,0.683694,"[2071, 2072]",1.0,False
18281,Adi Keyh,14.8333,39.3667,Eritrea,13061.0,0.258890,0.687079,[3967],0.0,False
